In this tutorial we will train and test a linear regressor. This is something you have already done in part 3 of the course document. So, I will miss out all of the extra notes and just get to business.

In [59]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/1122131uhi/dataAnalytics/master/tutorial2lineardata.csv', index_col=0, )

In [60]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

    day  temp  wdsp  NUM_TRIPS
2     6  20.0   7.4   0.631126
3     7  30.0  10.8   0.724951
6     3  30.4   5.7   0.716044
8     5  37.9  16.4   0.799994
9     6  25.7  13.8   0.872611
10    7  21.6   6.9   0.809792


In [61]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [62]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.9.2


In [64]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration, in this case, I have gone for day, temp, wdsp
df_input_data = [df["day"], df["temp"], df["wdsp"], df["NUM_TRIPS"]] 
# create headers for our new dataframe. These should correlate with the above.
df_input_headers = ["day", "temp", "wdsp", "NUM_TRIPS"] 
# create a final dataframe using our new dataframe and headers. 
df_input = pd.concat(df_input_data, axis=1, keys=df_input_headers) 

In [65]:
# construct a training set for runnign through the model and a test set, we do this by using sample with 0.8 for an 80% training set and 20% for test.
training_set = df_input.sample(frac=0.8, random_state=0)
test_set = df_input.drop(training_set.index)

In [66]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features = training_set.copy()
test_features = test_set.copy()

training_labels = training_features.pop('NUM_TRIPS')
test_labels = test_features.pop('NUM_TRIPS')

In [67]:
# Here I have put in a scale factor and divided by it. In this dataset, I had already normalised and thus it is 1. However, 600000 is what would make sense based on the data here and we can use this later when testing our model.. 
training_labels = training_labels/SCALE_NUM_TRIPS
test_labels = test_labels/SCALE_NUM_TRIPS

In [80]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [84]:
# I have decided to call the model, model_1. We add our normaliser and we are expecting a single output. 
model_1 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

In [86]:
# more boiler plate for creating a sequential model, we need an optimiser and loss parameter. Here we are going to be using the mean absolute error MAE
model_1.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [87]:
# now we are going to fit the model where we require the training features and labels. We will run it 100 times i.e. epochs and we have applied a further 20% validation split. 

%%time
history = model_1.fit(
    training_features,
    training_labels,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 5.88 s, sys: 302 ms, total: 6.18 s
Wall time: 6.38 s


In [88]:
# Now, we will evaluate our model using the test features and labels. 
mean_absolute_error_model_1 = model_1.evaluate(
    test_features,
    test_labels, verbose=0)

In [89]:
# The mean absolute error of the model can be printed out. Remember, we want to minimise this. Perhaps a model with just day and NUM_TRIPS would be better. It will also vary on each training run due to randomisation.
print(mean_absolute_error_model_1)

0.05905775725841522


In [91]:
# we create a custom dataframe with 3 values per feature. 
input_1 = pd.DataFrame.from_dict(data = 
				{
            'day' : [1,1,1],
            'temp' : [61.8, 31.2, 77.0],
            'wdsp' : [5.0, 3.0, 8.0]
        })

In [93]:
# next we can check this out, you can multiply by 600000 to get more realistic NUM_TRIPS values. 
linear_day_predictions_1 = model_1.predict(input_1[:3])*SCALE_NUM_TRIPS # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data
print(linear_day_predictions_1)

1/1 [==============================] - 0s 18ms/step
[[0.6537206]
 [0.6791478]
 [0.6391644]]


From test inputs we have used, the first element in the array here is similar to this actual data:

"165",1,62.4,5.6,0.668458757342322

We can see that the temperature is slightly higher in the test data (which means less trips, but slightly higher wind means more trips. So, the difference between (actual) 0.668 and (predicted) 0.576 (rounded to 3 significant figures) seems reasonable.

Similarly with the second:

"389",1,26.6,3.1,0.763954173062719, which has higher number of trips due to a lower temperature and also with a slightly higher wind speed.

And with the third:

"571",1,77.2,8.4,0.724652060408235

The last prediction with the higher temperature seems to punish the values more.

In [94]:
# same as above
input_2 = pd.DataFrame.from_dict(data = 
				{
            'day' : [6,6,6],
            'temp' : [61.8, 31.2, 77.0],
            'wdsp' : [5.0, 3.0, 8.0]
        })

In [95]:
linear_day_predictions_2 = model_1.predict(input_2[:3])*SCALE_NUM_TRIPS # essentially 600000 in this instance would give back realistic numbers based on the TAXI_TRIPS data
print(linear_day_predictions_2)

1/1 [==============================] - 0s 26ms/step
[[0.8802397]
 [0.9056669]
 [0.8656835]]


This test uses day 6 (Friday) instead of day 1 (Sunday) which shows higher number of trips. The other values were left the same.

Things to think about for the assignment. Make a validation set i.e. 5% of the data (or maybe more). This should be used for this type of testing. My values are simply made up.

You should also remember to use different models with different data. In this case, I would maybe take each input valuable separately and make a regression model for each, then different variations i.e. any 2.

Remember, you need to write up your results in the assignment. 